In [1]:
#インポート文
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.support.ui import Select,WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

import os
import time
import requests

import re
from datetime import datetime

In [2]:
#保存するディレクトリ
dir = 'takamatsu/horsepage/2022'
threshold_date = datetime(2022, 3, 20)

In [3]:
#raceのURL
RACE_URL = "https://db.netkeiba.com/race/202207020611/"

予測したいレースに関連する馬をスクレイピングするコード

In [4]:
#ドライバーの設定
driver = webdriver.Chrome(ChromeDriverManager().install())
wait = WebDriverWait(driver,10)

C:\Users\kawam\AppData\Local\Temp\ipykernel_1724\2084027807.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [5]:
#ドライバー起動
driver.get(RACE_URL)
time.sleep(1)
wait.until(EC.presence_of_all_elements_located)

<function selenium.webdriver.support.expected_conditions.presence_of_all_elements_located.<locals>._predicate(driver)>

In [6]:
horse_href_list=[]
all_rows = driver.find_element(By.CLASS_NAME,'race_table_01').find_elements(By.TAG_NAME,"tr")
for row in range(1, len(all_rows)):
    horse_href=all_rows[row].find_elements(By.TAG_NAME,"td")[3].find_element(By.TAG_NAME,"a").get_attribute("href")
    horse_href_list.append(horse_href)

In [7]:
HTML_DIR = "html/"+dir
if not os.path.isdir(HTML_DIR):
    os.makedirs(HTML_DIR)
        
for url in horse_href_list:
    list = url.split("/")
    horse_id = list[-2]
    save_file_path = HTML_DIR+"/"+horse_id+'.html'
    response = requests.get(url)
    response.encoding = response.apparent_encoding
    html = response.text
    time.sleep(5)
    with open(save_file_path, 'w') as file:
        file.write(html)

In [8]:
horse_data_info_columns=[
    'horse_id',
    'bday',
    'tame_id',
    'owner_id',
    'producer_id',
    'production area',
    'auction price',
    'winnings',
    'lifetime record',
    'wined race title',
    'inbreeding-1',
    'inbreeding-2',
    'father',
    'faths father',
    'faths mother',
    'mother',
    'moths father',
    'moths mother'
]

In [9]:
horse_race_columns=["date",
                    "whereracecourse",
                    "weather",
                    "race_round",
                    "race_title",
                    "race_id",
                    "total_horse_number",
                    "frame_number",
                    "horse_number",
                    "odds",
                    "popular",
                    "rank",
                    "rider_id",
                    "horse_wight",
                    "distance",
                    "groud_status",
                    'goal_time',
                    'goal_time_dif',
                    'half_way_rank',
                    'pace',
                    'last_time',
                    'horse_weight',
                    'runner-up-horse-id',
                    'prize'
                    ]
print(len(horse_race_columns))

24


In [10]:
def get_horse_data_by_html(horse_id, html):
    horse_list = [horse_id]
    horse_race_list = []
    soup = BeautifulSoup(html, 'html.parser')
    horse_tables = soup.find("table", class_="db_prof_table")
    horse_table = horse_tables.findAll("td")
    #for i in range(10):
    horse_list.append(horse_table[0].get_text())
    horse_list.append(horse_table[1].find('a').get('href').split("/")[-2])
    horse_list.append(horse_table[2].find('a').get('href').split("/")[-2])
    try:
        horse_list.append(horse_table[3].find('a').get('href').split("/")[-2])
    except AttributeError:
        horse_list.append(None)
    horse_list.append(horse_table[4].get_text())
    horse_list.append(horse_table[5].get_text())
    horse_list.append(horse_table[6].get_text())
    horse_list.append(horse_table[7].get_text())
    horse_list.append(horse_table[8].find('a').get('href').split("/")[-2])
    
    # horse_table[8]からすべてのリンクのURLの一部を取得する
    url_parts = [link.get('href').split("/")[-2] for link in horse_table[9].find_all('a')]

    # url_partsの要素数が2未満の場合、残りの要素をNaNで埋める
    while len(url_parts) < 2:
        url_parts.append(np.nan)

    # horse_listにurl_partsを追加する
    horse_list.extend(url_parts)
    
    #horse_list.append(horse_table[9].get_text())
    blood_tables = soup.find("table", class_="blood_table")
    blood_table = blood_tables.findAll("td")
    #for i in range(6):
    horse_list.append(blood_table[0].find('a').get('href').split("/")[-2])
    horse_list.append(blood_table[1].find('a').get('href').split("/")[-2])
    horse_list.append(blood_table[2].find('a').get('href').split("/")[-2])
    horse_list.append(blood_table[3].find('a').get('href').split("/")[-2])
    horse_list.append(blood_table[4].find('a').get('href').split("/")[-2])
    horse_list.append(blood_table[5].find('a').get('href').split("/")[-2])    
    
    
    
    
    
#horse_race
    horse_race_tmp_df = pd.DataFrame()

    horse_tables = soup.find("table", class_="db_h_race_results")
    horse_trs = horse_tables.findAll("tr")
    for index, horse_tr in enumerate(horse_trs):
        if index == 0:
            continue
        horse_race_list = []
        horse_ra_se = pd.Series()

        horse_tds = horse_tr.findAll("td")
        horse_race_list.append(horse_tds[0].get_text())
        horse_race_list.append(horse_tds[1].get_text())
        horse_race_list.append(horse_tds[2].get_text())
        horse_race_list.append(horse_tds[3].get_text())
        horse_race_list.append(horse_tds[4].get_text())
        horse_race_list.append(horse_tds[4].find('a').get('href').split("/")[-2])
        horse_race_list.append(horse_tds[6].get_text())
        horse_race_list.append(horse_tds[7].get_text())
        horse_race_list.append(horse_tds[8].get_text())
        horse_race_list.append(horse_tds[9].get_text())
        horse_race_list.append(horse_tds[10].get_text())
        horse_race_list.append(horse_tds[11].get_text())
        try:
            horse_race_list.append(horse_tds[12].find('a').get('href').split("/")[-2])
        except AttributeError:
            horse_race_list.append(None)
        horse_race_list.append(horse_tds[13].get_text())
        horse_race_list.append(horse_tds[14].get_text())
        horse_race_list.append(horse_tds[15].get_text())
        horse_race_list.append(horse_tds[17].get_text())
        horse_race_list.append(horse_tds[18].get_text())
        horse_race_list.append(horse_tds[20].get_text())
        horse_race_list.append(horse_tds[21].get_text())
        horse_race_list.append(horse_tds[22].get_text())
        horse_race_list.append(horse_tds[23].get_text())
        try:
            horse_race_list.append(horse_tds[26].find('a').get('href').split("/")[-2])
        except AttributeError:
            horse_race_list.append(None)
        horse_race_list.append(horse_tds[27].get_text())
        horse_ra_se = pd.Series(horse_race_list, index=horse_race_columns,dtype='object')
        horse_race_tmp_df = pd.concat([horse_race_tmp_df, horse_ra_se.to_frame().T], ignore_index=True)
#     horse_race_tmp_df = pd.DataFrame(horse_race_list, columns=horse_race_columns)
    horse_race_tmp_df.loc[:, 'horse_id'] = horse_id
    print(horse_race_tmp_df.shape)
    
    return horse_list , horse_race_tmp_df

In [11]:
CSV_DIR = "csv/"+dir
if not os.path.isdir(CSV_DIR):
    os.makedirs(CSV_DIR)
horse_info_csv = CSV_DIR+"/horse-info.csv"
horse_race_csv = CSV_DIR+"/horse-race.csv"
horse_info_df = pd.DataFrame(columns=horse_data_info_columns)
horse_race_df = pd.DataFrame()
if os.path.isdir(HTML_DIR):
    file_list = os.listdir(HTML_DIR)
    for file_name in file_list:
        with open(HTML_DIR+"/"+file_name, "r") as f:
            html = f.read()
            list = file_name.split(".")
            horse_id = list[-2]
            horse_list , horse_race_tmp_df = get_horse_data_by_html(horse_id, html) 
            horse_se = pd.Series(horse_list, index=horse_info_df.columns)
            horse_info_df = pd.concat([horse_info_df, horse_se.to_frame().T], ignore_index=True)
            horse_race_df = pd.concat([horse_race_df, horse_race_tmp_df], axis=0, ignore_index=True)       

C:\Users\kawam\AppData\Local\Temp\ipykernel_1724\2637880374.py:55: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  horse_ra_se = pd.Series()
C:\Users\kawam\AppData\Local\Temp\ipykernel_1724\2637880374.py:55: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  horse_ra_se = pd.Series()
C:\Users\kawam\AppData\Local\Temp\ipykernel_1724\2637880374.py:55: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  horse_ra_se = pd.Series()
C:\Users\kawam\AppData\Local\Temp\ipykernel_1724\2637880374.py:55: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  ho

C:\Users\kawam\AppData\Local\Temp\ipykernel_1724\2637880374.py:55: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  horse_ra_se = pd.Series()
C:\Users\kawam\AppData\Local\Temp\ipykernel_1724\2637880374.py:55: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  horse_ra_se = pd.Series()
C:\Users\kawam\AppData\Local\Temp\ipykernel_1724\2637880374.py:55: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  horse_ra_se = pd.Series()
C:\Users\kawam\AppData\Local\Temp\ipykernel_1724\2637880374.py:55: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  ho

(62, 25)


C:\Users\kawam\AppData\Local\Temp\ipykernel_1724\2637880374.py:55: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  horse_ra_se = pd.Series()
C:\Users\kawam\AppData\Local\Temp\ipykernel_1724\2637880374.py:55: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  horse_ra_se = pd.Series()
C:\Users\kawam\AppData\Local\Temp\ipykernel_1724\2637880374.py:55: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  horse_ra_se = pd.Series()
C:\Users\kawam\AppData\Local\Temp\ipykernel_1724\2637880374.py:55: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  ho

(28, 25)


...

C:\Users\kawam\AppData\Local\Temp\ipykernel_1724\2637880374.py:55: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  horse_ra_se = pd.Series()
C:\Users\kawam\AppData\Local\Temp\ipykernel_1724\2637880374.py:55: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  horse_ra_se = pd.Series()
C:\Users\kawam\AppData\Local\Temp\ipykernel_1724\2637880374.py:55: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  horse_ra_se = pd.Series()
C:\Users\kawam\AppData\Local\Temp\ipykernel_1724\2637880374.py:55: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  ho

(16, 25)


C:\Users\kawam\AppData\Local\Temp\ipykernel_1724\2637880374.py:55: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  horse_ra_se = pd.Series()
C:\Users\kawam\AppData\Local\Temp\ipykernel_1724\2637880374.py:55: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  horse_ra_se = pd.Series()
C:\Users\kawam\AppData\Local\Temp\ipykernel_1724\2637880374.py:55: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  horse_ra_se = pd.Series()
C:\Users\kawam\AppData\Local\Temp\ipykernel_1724\2637880374.py:55: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  ho

(13, 25)
(14, 25)


C:\Users\kawam\AppData\Local\Temp\ipykernel_1724\2637880374.py:55: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  horse_ra_se = pd.Series()
C:\Users\kawam\AppData\Local\Temp\ipykernel_1724\2637880374.py:55: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  horse_ra_se = pd.Series()
C:\Users\kawam\AppData\Local\Temp\ipykernel_1724\2637880374.py:55: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  horse_ra_se = pd.Series()
C:\Users\kawam\AppData\Local\Temp\ipykernel_1724\2637880374.py:55: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  ho

In [12]:
horse_race_df

,date,whereracecourse,weather,race_round,race_title,race_id,total_horse_number,frame_number,horse_number,odds,...,groud_status,goal_time,goal_time_dif,half_way_rank,pace,last_time,horse_weight,runner-up-horse-id,prize,horse_id
0,2022/12/24,6阪神7,晴,11,阪神C(G2),202209060711,18,1,2,368.2,...,良,1:20.4,0.2,13-13,33.4-35.5,34.8,532(+6),2015103057,670.0,2014102625
1,2022/12/04,5中山2,曇,11,ラピスラズリS(L),202206050211,16,4,8,123.7,...,良,1:08.6,0.5,14-14,33.7-34.4,33.9,526(+2),2016110119,,2014102625
2,2022/11/27,5阪神8,晴,12,京阪杯(G3),202209050812,16,8,15,206.6,...,良,1:08.8,1.6,6-7,33.3-33.9,34.5,524(0),2019106526,,2014102625
3,2022/10/30,4新潟6,曇,11,ルミエールオータムD(L),202204040611,18,7,15,51.8,...,稍,0:55.8,0.7,15,32.8-32.9,32.3,524(-2),2018110120,,2014102625
4,2022/10/09,4阪神2,雨,11,夕刊フジ杯オパールS(L),202209040211,16,8,15,95.7,...,稍,1:09.7,1.1,15-14,33.8-34.8,34.6,526(-2),2019106526,,2014102625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448,2021/03/06,1阪神7,曇,11,チューリップ賞(G2),202109010711,12,1,1,1.6,...,稍,1:33.8,0.0,4-1,36.3-34.8,34.8,462(+4),2018110004,"3,704.0",2018105069
449,2020/12/13,6阪神4,曇,11,阪神ジュベナイルF(G1),202009060411,18,8,18,4.7,...,良,1:33.3,0.2,12-8,34.9-34.4,34.0,458(0),2018105233,980.0,2018105069
450,2020/11/07,5阪神1,曇,11,KBSファンタジーS(G3),202009050111,12,7,10,2.5,...,良,1:20.1,-0.1,4-4,34.1-34.8,34.5,458(-2),2018101067,"2,944.8",2018105069
451,2020/09/06,2小倉8,雨,11,小倉2歳S(G3),202010020811,10,7,8,6.3,...,重,1:09.6,-0.2,5-4,33.9-35.7,35.1,460(-2),2018105296,"3,135.7",2018105069


### cleaning

##### horse_info_df

In [13]:
horse_info_df.head(1)

,horse_id,bday,tame_id,owner_id,producer_id,production area,auction price,winnings,lifetime record,wined race title,inbreeding-1,inbreeding-2,father,faths father,faths mother,mother,moths father,moths mother
0,2014102625,2014年3月27日,01142,177033,733363,日高町,\n-\n,"\n\n\n2億3,695万円 (中央)\n\n",62戦7勝 [7-8-4-43],201806050211,2013101042,2016103788,1997110025,000a00193b,000a00abee,2007104165,1993109188,000a00fa5a


##### horse_race

In [14]:
horse_race_df.head(1)

,date,whereracecourse,weather,race_round,race_title,race_id,total_horse_number,frame_number,horse_number,odds,...,groud_status,goal_time,goal_time_dif,half_way_rank,pace,last_time,horse_weight,runner-up-horse-id,prize,horse_id
0,2022/12/24,6阪神7,晴,11,阪神C(G2),202209060711,18,1,2,368.2,...,良,1:20.4,0.2,13-13,33.4-35.5,34.8,532(+6),2015103057,670.0,2014102625


In [15]:
horse_race_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 453 entries, 0 to 452
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   date                453 non-null    object
 1   whereracecourse     453 non-null    object
 2   weather             453 non-null    object
 3   race_round          453 non-null    object
 4   race_title          453 non-null    object
 5   race_id             453 non-null    object
 6   total_horse_number  453 non-null    object
 7   frame_number        453 non-null    object
 8   horse_number        453 non-null    object
 9   odds                453 non-null    object
 10  popular             453 non-null    object
 11  rank                453 non-null    object
 12  rider_id            453 non-null    object
 13  horse_wight         453 non-null    object
 14  distance            453 non-null    object
 15  groud_status        453 non-null    object
 16  goal_time           453 no

##### date

In [16]:
horse_race_df['date'] = pd.to_datetime(horse_race_df['date'], format='%Y/%m/%d')

In [17]:
horse_race_df.drop(horse_race_df[horse_race_df['date'] >= threshold_date].index, inplace=True)

##### race_title

In [18]:
# 'race_title' 列から () の中身を抽出する
horse_race_df['race_type'] = horse_race_df['race_title'].str.extract('\((.*?)\)')

# 'race_type' 列の欠損値を空文字列に置き換える
horse_race_df['race_type'].fillna('', inplace=True)

##### odds

In [19]:
#数字以外の文字を削除
horse_race_df['odds'] = horse_race_df['odds'].apply(lambda x: re.sub('[^0-9\.]', '', x))
#空白を削除
horse_race_df['odds'] = horse_race_df['odds'].replace('', np.nan)
#欠損値の行を削除
horse_race_df = horse_race_df.dropna(subset=['odds'])

In [20]:
horse_race_df['odds'] = horse_race_df['odds'].astype(float)

C:\Users\kawam\AppData\Local\Temp\ipykernel_1724\2488911290.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  horse_race_df['odds'] = horse_race_df['odds'].astype(float)


##### rank

In [21]:
#数字以外の文字を含む列を削除
horse_race_df = horse_race_df[pd.to_numeric(horse_race_df['rank'], errors='coerce').notnull()]
#数値に変換
horse_race_df['rank'] = horse_race_df['rank'].astype(int)

C:\Users\kawam\AppData\Local\Temp\ipykernel_1724\4046826464.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  horse_race_df['rank'] = horse_race_df['rank'].astype(int)


##### distance

In [22]:
#文字と数値に分離させる
horse_race_df[['surface', 'distance']] = horse_race_df['distance'].str.extract('(\D+)(\d+)')
# 'distance' 列を整数値に変換する
horse_race_df['distance'] = horse_race_df['distance'].astype(int)

##### goal_time

In [23]:
horse_race_df['goal_time'].unique()

array(['1:08.5', '1:09.5', '1:09.4', '1:21.2', '1:08.4', '0:55.4',
       '1:09.6', '1:09.8', '1:08.8', '1:24.0', '1:09.3', '1:26.4',
       '1:10.2', '1:09.7', '1:11.9', '1:07.8', '1:23.2', '1:36.8',
       '1:23.1', '1:10.7', '1:10.6', '1:20.4', '1:11.0', '1:08.1',
       '1:08.7', '1:19.8', '1:07.7', '1:21.8', '1:21.7', '1:33.8',
       '1:07.6', '1:20.7', '1:20.8', '1:22.0', '1:24.1', '1:09.0',
       '1:21.3', '1:22.4', '1:20.9', '1:33.9', '1:36.3', '1:22.8',
       '1:10.4', '1:08.3', '1:22.3', '1:09.2', '1:10.3', '1:20.3',
       '1:12.1', '1:36.6', '1:38.5', '1:39.4', '1:55.1', '1:39.2',
       '1:37.9', '1:41.3', '1:40.2', '2:03.0', '2:17.6', '2:01.9',
       '1:35.9', '1:31.4', '1:30.8', '1:10.9', '1:09.1', '1:10.8',
       '1:28.8', '1:34.8', '1:35.0', '1:36.7', '1:22.7', '1:19.9',
       '1:33.0', '1:10.0', '1:12.9', '1:07.5', '1:34.2', '1:19.6',
       '1:31.9', '1:22.2', '1:21.5', '1:20.2', '1:33.5', '1:33.3',
       '1:21.4', '1:34.9', '1:20.0', '1:08.2', '1:35.3', '1:39

In [24]:
#数字以外の文字を削除
horse_race_df['goal_time'] = horse_race_df['goal_time'].apply(lambda x: re.sub(r"[^0-9:.]", "", x))
#空白を削除
horse_race_df['goal_time'] = horse_race_df['goal_time'].replace('', np.nan)
#欠損値の行を削除
horse_race_df = horse_race_df.dropna(subset=['goal_time'])
#数値に変換
def to_seconds(time_str):
    minutes, seconds = map(float, time_str.split(':'))
    return minutes * 60 + seconds

horse_race_df['goal_time'] = horse_race_df['goal_time'].apply(to_seconds)

##### goal_time_dif

In [25]:
#数字以外の文字を削除
horse_race_df['goal_time_dif'] = horse_race_df['goal_time_dif'].apply(lambda x: re.sub('[^0-9\.]', '', x))
#空白を削除
horse_race_df['goal_time_dif'] = horse_race_df['goal_time_dif'].replace('', np.nan)
horse_race_df['goal_time_dif'] = horse_race_df['goal_time_dif'].astype(float)

In [26]:
horse_race_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 354 entries, 9 to 452
Data columns (total 27 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   date                354 non-null    datetime64[ns]
 1   whereracecourse     354 non-null    object        
 2   weather             354 non-null    object        
 3   race_round          354 non-null    object        
 4   race_title          354 non-null    object        
 5   race_id             354 non-null    object        
 6   total_horse_number  354 non-null    object        
 7   frame_number        354 non-null    object        
 8   horse_number        354 non-null    object        
 9   odds                354 non-null    float64       
 10  popular             354 non-null    object        
 11  rank                354 non-null    int32         
 12  rider_id            354 non-null    object        
 13  horse_wight         354 non-null    object        

In [27]:
horse_info_df.to_csv(horse_info_csv, header=True, index=False)
horse_race_df.to_csv(horse_race_csv, header=True, index=False)

##### 以下メモ

In [28]:
# # データを読み込む
# data = pd.read_csv('data.csv')

# # 日付でソートする
# data = data.sort_values('date')

# # 同じ ID を持つデータをグループ化する
# grouped_data = data.groupby('ID')

# # 各グループの最初のデータを選択する
# latest_data = grouped_data.last()